In [8]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import json
import yaml
import ast
from pathlib import Path
from functools import partial
from tqdm.notebook import tqdm
from IPython.display import Audio
import torch
import torchaudio
from torchaudio.transforms import MelSpectrogram
import librosa
from torch.utils.data import Dataset, DataLoader, ConcatDataset, Subset
from fusanet_utils.datasets.simulated import SimulatedPoliphonic
from fusanet_utils.datasets.fusa import FUSA_dataset

class Vicente(Dataset):
    def __init__(self, categories):
        self.file_paths = []
        self.meta = pd.read_csv('../../datasets/Vicente/Metadata.csv', 
                                encoding='latin-1', sep=';', header=0, comment='#', parse_dates=[4])
        self.labels = []
        self.folder = []
        for file in Path('../../datasets/Vicente/').rglob('*.wav'):
            tag = self.match_ids(file)["Tag 1"].apply(lambda x: x.split(':')[0] if isinstance(x, str) else None)
            if len(tag) == 0:
                #print(id, file, tag)
                continue
            self.labels.append(tag.item().lower())
            #self.labels.append(tag.item())
            self.file_paths.append(file)
            self.folder.append(file.parent.stem)
        self.categories = categories
    
    def match_ids(self, file_name):
        id = file_name.stem
        if id[-3:] == '(1)':
            id = id[:-3]
        return self.meta.loc[self.meta['Id'] == id]         
        
    def __getitem__(self, idx):
        #return self.file_paths[idx], 'dog'
        return self.file_paths[idx], self.labels[idx]
    
    def __len__(self):
        return len(self.file_paths)

In [9]:
import pickle
with open('UrbanSound_ESC-PANN-tag-pink-noise-pretrained.pkl', 'rb') as f:
    vicente_results = pickle.load(f)
    
th = 0.5
summarized_result = []
for name, percentages in vicente_results.items():
    summarized_result.append((percentages > th).sum()/12)

In [11]:
dataset = Vicente(categories=vicente_df.columns)
path, _ = dataset[116]
meta = dataset.match_ids(path)
display(meta)

,Id,Tag 1,Tag 2,Tag 3,Fecha y hora
44,1654237620_1654237680,animal/dog:1.0,no probability score,no probability score,2022-06-03 06:27:00


In [12]:
names = list(dataset.meta.Id)

In [13]:
fusa_metadata = pd.read_csv('../SPASS_evaluation/real-data/fusa_metadata_2022_08_26__20_44_24.csv')
fusa_metadata

/tmp/ipykernel_60174/2695235872.py:1: DtypeWarning: Columns (42) have mixed types. Specify dtype option on import or set low_memory=False.
  fusa_metadata = pd.read_csv('../SPASS_evaluation/real-data/fusa_metadata_2022_08_26__20_44_24.csv')


,description,_id,name,backup_state,duration,size,date,dbALeq,dbAmax,dbAmin,...,probability_13_class,probability_13_value,probability_14_class,probability_14_value,probability_15_class,probability_15_value,probability_16_class,probability_16_value,probability_17_class,probability_17_value
0,Grabación CAPTA desde la estacion Campus Miraf...,62800707ae49cd33b56d4960,1652483228_1652483288,deleted,60,0,2022-05-13 23:07:08,63.400000,67.3,43.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Grabación CAPTA desde la estacion Campus Miraf...,62800716ae49cd33b56d4961,1652485590_1652485650,deleted,60,0,2022-05-13 23:46:30,63.000000,69.3,39.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Grabación CAPTA desde la estacion Campus Miraf...,62800730ae49cd33b56d4963,1652472437_1652472497,deleted,60,0,2022-05-13 20:07:17,65.000000,68.8,52.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Grabación CAPTA desde la estacion Campus Miraf...,62800740ae49cd33b56d4964,1652539573_1652539633,deleted,60,0,2022-05-14 14:46:13,59.100000,67.8,38.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Grabación CAPTA desde la estacion Campus Miraf...,62800759ae49cd33b56d4966,1652450378_1652450438,deleted,60,0,2022-05-13 13:59:38,63.300000,68.6,46.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105918,Grabación CAPTA desde la estacion Capta 004,62bba45378257cc153cd740d,1656183996_1656184026,in_cloud,30,5760044,2022-06-25 19:06:36,55.822014,58.4,53.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
105919,Grabación CAPTA desde la estacion Capta 004,62bba47078257cc153cd740e,1656184027_1656184057,in_cloud,30,5760044,2022-06-25 19:07:07,56.284856,58.7,54.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
105920,Grabación CAPTA desde la estacion Capta 004,62bba4a178257cc153cd7410,1656184058_1656184088,in_cloud,30,5760044,2022-06-25 19:07:38,57.967724,60.0,55.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
105921,Grabación CAPTA desde la estacion Capta 004,62bba4d578257cc153cd7412,1656184089_1656184119,in_cloud,30,5760044,2022-06-25 19:08:09,56.720221,58.0,55.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
fusa_metadata

,description,_id,name,backup_state,duration,size,date,dbALeq,dbAmax,dbAmin,...,probability_13_class,probability_13_value,probability_14_class,probability_14_value,probability_15_class,probability_15_value,probability_16_class,probability_16_value,probability_17_class,probability_17_value
0,Grabación CAPTA desde la estacion Campus Miraf...,62800707ae49cd33b56d4960,1652483228_1652483288,deleted,60,0,2022-05-13 23:07:08,63.400000,67.3,43.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Grabación CAPTA desde la estacion Campus Miraf...,62800716ae49cd33b56d4961,1652485590_1652485650,deleted,60,0,2022-05-13 23:46:30,63.000000,69.3,39.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Grabación CAPTA desde la estacion Campus Miraf...,62800730ae49cd33b56d4963,1652472437_1652472497,deleted,60,0,2022-05-13 20:07:17,65.000000,68.8,52.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Grabación CAPTA desde la estacion Campus Miraf...,62800740ae49cd33b56d4964,1652539573_1652539633,deleted,60,0,2022-05-14 14:46:13,59.100000,67.8,38.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Grabación CAPTA desde la estacion Campus Miraf...,62800759ae49cd33b56d4966,1652450378_1652450438,deleted,60,0,2022-05-13 13:59:38,63.300000,68.6,46.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105918,Grabación CAPTA desde la estacion Capta 004,62bba45378257cc153cd740d,1656183996_1656184026,in_cloud,30,5760044,2022-06-25 19:06:36,55.822014,58.4,53.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
105919,Grabación CAPTA desde la estacion Capta 004,62bba47078257cc153cd740e,1656184027_1656184057,in_cloud,30,5760044,2022-06-25 19:07:07,56.284856,58.7,54.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
105920,Grabación CAPTA desde la estacion Capta 004,62bba4a178257cc153cd7410,1656184058_1656184088,in_cloud,30,5760044,2022-06-25 19:07:38,57.967724,60.0,55.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
105921,Grabación CAPTA desde la estacion Capta 004,62bba4d578257cc153cd7412,1656184089_1656184119,in_cloud,30,5760044,2022-06-25 19:08:09,56.720221,58.0,55.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
ids = []

In [16]:
for row in fusa_metadata.iterrows():
    if row[1]['name'] in names:
        print(row[1]._id, row[1]['name'])
        ids.append(row[1]._id)

628f8fbbde45dbd74506f8a3 1653306233_1653306293
628f91d2de45dbd74506f8b5 1653323280_1653323340
628fa35dde45dbd74506f97c 1653357451_1653357511
628fa679de45dbd74506f9ad 1653360419_1653360479
628fa797de45dbd74506f9bf 1653361511_1653361571
628fb10cde45dbd74506fa56 1653373862_1653373922
628fb697de45dbd74506faa7 1653385560_1653385620
628fbb08de45dbd74506faef 1653393927_1653393987
628fbf5cde45dbd74506fb2b 1653409372_1653409432
628fbf7ede45dbd74506fb2d 1653409736_1653409796
628fcb0ede45dbd74506fb7b 1653428341_1653428401
628fd778de45dbd74506fbcb 1653446828_1653446888
628fe21fde45dbd74506fc0f 1653460321_1653460381
628fe61cde45dbd74506fc26 1653472020_1653472080
628ff91dde45dbd74506fca2 1653514199_1653514259
6291056fde45dbd74506fe12 1653617629_1653617689
62910b1bde45dbd74506fe83 1653626700_1653626760
62910cabde45dbd74506fe9f 1653633123_1653633183
629113e5de45dbd74506ff00 1653662863_1653662923
6293f073de45dbd745070058 1653717766_1653717826
629637efde45dbd7450702ce 1653994181_1653994241
629a346bde45d

In [17]:
ids

['628f8fbbde45dbd74506f8a3',
 '628f91d2de45dbd74506f8b5',
 '628fa35dde45dbd74506f97c',
 '628fa679de45dbd74506f9ad',
 '628fa797de45dbd74506f9bf',
 '628fb10cde45dbd74506fa56',
 '628fb697de45dbd74506faa7',
 '628fbb08de45dbd74506faef',
 '628fbf5cde45dbd74506fb2b',
 '628fbf7ede45dbd74506fb2d',
 '628fcb0ede45dbd74506fb7b',
 '628fd778de45dbd74506fbcb',
 '628fe21fde45dbd74506fc0f',
 '628fe61cde45dbd74506fc26',
 '628ff91dde45dbd74506fca2',
 '6291056fde45dbd74506fe12',
 '62910b1bde45dbd74506fe83',
 '62910cabde45dbd74506fe9f',
 '629113e5de45dbd74506ff00',
 '6293f073de45dbd745070058',
 '629637efde45dbd7450702ce',
 '629a346bde45dbd74507036c',
 '629a34b2de45dbd745070371',
 '629a34cede45dbd745070373',
 '629a34f7de45dbd745070376',
 '629a3522de45dbd745070378',
 '629a3531de45dbd745070379',
 '629a3542de45dbd74507037a',
 '629a355ade45dbd74507037b',
 '629a3578de45dbd74507037d',
 '629a35cede45dbd745070382',
 '629a35e3de45dbd745070384',
 '629a3612de45dbd745070387',
 '629a3642de45dbd74507038b',
 '629a3675de45

In [19]:
from fusanet_utils.datasets.aumilab import AUMILAB
from fusanet_utils.transforms import Collate_and_transform

experiment_path = Path('../../experiments/Poliphonic-PANN-sed-no-pretrained-valid-loss/')
#experiment_path = Path('../../experiments/Poliphonic-PANN-sed-pink-noise-2/')
print(experiment_path)

categories = json.load(open(str(experiment_path / 'index_to_name.json')))
params = yaml.safe_load(open(str(experiment_path / 'params.yaml')))

dataset = AUMILAB('../..', use_original_labels=True, categories=categories.values())

fusa_dataset = FUSA_dataset(ConcatDataset([dataset]), params)
fusa_loader = DataLoader(fusa_dataset, batch_size=5, shuffle=False, pin_memory=True, num_workers=2,
                         collate_fn=Collate_and_transform(params["features"]))

../../experiments/Poliphonic-PANN-sed-no-pretrained-valid-loss


In [20]:
len(dataset.file_list)

14804

In [21]:
dataset.file_list[0].parts[-1]

'1653661967_1653662027_0.wav'

In [23]:
cruce = []
c = 0
for filename in dataset.file_list:
    if filename.parts[-1].split('.')[0] in names:
        cruce.append(filename.parts[-1].split('.')[0])
        print(filename)
        print(filename.parts[-1].split('.')[0])
        c += 1
print(c)

0


In [24]:
len(cruce)

0

In [25]:
vicente_subset = []
for a in dataset:
    if a[0].parts[-1].split('.')[0] in cruce:
        print(a[0].parts[-1].split('.')[0], list(a[1].index))
        for i in list(a[1].index):
            vicente_subset.append(i)

In [26]:
vicente_subset

[]

In [27]:
subset_cruce = Subset(dataset, vicente_subset)
subset_cruce

In [28]:
subset_cruce

In [29]:
subset_cruce.categories = categories.values()
fusa_dataset = FUSA_dataset(ConcatDataset([subset_cruce]), params)
fusa_loader = DataLoader(fusa_dataset, batch_size=11, pin_memory=True, num_workers=2,
                         collate_fn=Collate_and_transform(params["features"]))

In [30]:
len(fusa_dataset)

0

In [31]:
labels_model, file_names = [], []
for sample in tqdm(fusa_loader):
    print(sample)
    #labels_model.append(sample['label'].numpy())
    #file_names.append(sample['filename'])

0it [00:00, ?it/s]

In [32]:
file_names

[]

In [33]:
vicente_results

{'1653428341_1653428401.wav':     alerts/bells  alerts/horns  alerts/siren  animal/bird  animal/dog  \
 0       0.000394      0.094303      0.274811     0.000476    0.006185   
 1       0.000096      0.005661      0.811021     0.002447    0.009612   
 2       0.000103      0.000451      0.193802     0.093024    0.036153   
 3       0.001799      0.002857      0.093105     0.001209    0.013730   
 4       0.000005      0.007163      0.171164     0.000386    0.002893   
 5       0.000013      0.003472      0.753317     0.001877    0.005099   
 6       0.000537      0.000592      0.037170     0.000571    0.002242   
 7       0.000977      0.005579      0.469930     0.002688    0.042628   
 8       0.000074      0.016588      0.276479     0.000220    0.004493   
 9       0.000003      0.000007      0.100131     0.001281    0.015572   
 10      0.000129      0.001309      0.113365     0.002016    0.025005   
 11      0.000254      0.030565      0.208111     0.000269    0.006178   
 
     an

In [34]:
vicente_df

,alerts/bells,alerts/horns,alerts/siren,animal/bird,animal/dog,animal/others,environmental/others,environmental/rain,environmental/waves,environmental/wind,humans/others,mechanical/air_conditioner,mechanical/cutting,mechanical/digging,mechanical/explosives,mechanical/impact,mechanical/others,music/music,vehicles/airborne,vehicles/others
1653428341_1653428401.wav,0.0,0.0,0.166667,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.083333,0.0,0.0,0.000000,0.0,0.166667,0.0,0.0,0.0
1653460321_1653460381.wav,0.0,0.0,0.166667,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.083333,0.000000,0.0,0.0,0.416667,0.0,0.000000,0.0,0.0,0.0
1653472020_1653472080.wav,0.0,0.0,0.166667,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.416667,0.0,0.000000,0.0,0.0,0.0
1653626700_1653626760.wav,0.0,0.0,0.083333,0.0,0.083333,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.166667,0.0,0.000000,0.0,0.0,0.0
1653361511_1653361571.wav,0.0,0.0,0.250000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.333333,0.0,0.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1654270595_1654270655(1).wav,0.0,0.0,0.416667,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.166667,0.0,0.0,0.0
1654221272_1654221332(1).wav,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0
1654228064_1654228124(1).wav,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.5,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0
1654220421_1654220481.wav,0.0,0.0,0.166667,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.083333,0.0,0.000000,0.0,0.0,0.0


In [35]:
vicente_df = pd.DataFrame(summarized_result, index=list(vicente_results.keys()))
#display(vicente_df.head())
vicente_df.describe().T

,count,mean,std,min,25%,50%,75%,max
alerts/bells,117.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000
alerts/horns,117.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000
alerts/siren,117.0,0.078348,0.140679,0.0,0.0,0.0,0.166667,0.750000
animal/bird,117.0,0.000712,0.007704,0.0,0.0,0.0,0.000000,0.083333
animal/dog,117.0,0.095442,0.256212,0.0,0.0,0.0,0.000000,1.000000
animal/others,117.0,0.000712,0.007704,0.0,0.0,0.0,0.000000,0.083333
environmental/others,117.0,0.003561,0.025411,0.0,0.0,0.0,0.000000,0.250000
environmental/rain,117.0,0.243590,0.357117,0.0,0.0,0.0,0.416667,1.000000
environmental/waves,117.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000
environmental/wind,117.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000


In [38]:
mask = vicente_df['animal/dog'] > 0.5
idxs = vicente_df.loc[mask].index
if len(idxs) > 0:
    print(idxs)
    for idx in idxs:
        print(vicente_df.index.get_loc(idx))

Index(['1654237499_1654237559(1).wav', '1654237378_1654237438(1).wav',
       '1654394260_1654394320(1).wav', '1654237439_1654237499(1).wav',
       '1654237560_1654237620.wav', '1654237863_1654237923(1).wav',
       '1654237620_1654237680(1).wav', '1654238106_1654238166(1).wav',
       '1654237257_1654237317(1).wav', '1654237985_1654238045(1).wav',
       '1654238045_1654238105(1).wav', '1654225332_1654225392(1).wav'],
      dtype='object')
74
76
79
86
93
94
96
98
101
105
109
116
